## Deliverable 3. Create a Travel Itinerary Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [48]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df
#test = vacation_df.groupby("Hotel Name").sum()
#test

,City_ID,City,Country,Max Temp,description,Lat,Lng,Hotel Name
0,7,Bambous Virieux,MU,82.22,light rain,-20.3428,57.7575,Casa Tia Villa
1,19,Hobyo,SO,76.91,scattered clouds,5.3505,48.5268,Halane Hotel and Restaurent
2,20,Isangel,VU,84.63,broken clouds,-19.5500,169.2667,Tanna Lodge
3,30,Atuona,PF,77.31,light rain,-9.8000,-139.0333,Villa Enata
4,34,Kawalu,ID,84.00,overcast clouds,-7.3817,108.2082,KOSAN UNGU UNSIK
...,...,...,...,...,...,...,...,...
142,683,Katobu,ID,79.77,broken clouds,-4.9408,122.5276,La Ode Husein
143,684,Sawtell,AU,75.52,overcast clouds,-30.3833,153.1000,The Bungalow Sawtell
144,685,Ratnagiri,IN,78.21,clear sky,16.9833,73.3000,Hotel Landmark
145,686,Gua Musang,MY,84.09,overcast clouds,4.8823,101.9644,Mines Inn Hotel


In [49]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [50]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [142]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Arraial Do Cabo"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Arraial Do Cabo"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Sao Joao Da Barra"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Vila Velha"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Conceicao Da Barra"]

In [143]:
vacation_stop3

,City_ID,City,Country,Max Temp,description,Lat,Lng,Hotel Name
135,661,Conceicao Da Barra,BR,75.33,few clouds,-18.5933,-39.7322,Cricaré Praia Hotel


In [144]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple([vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]])
print(start)
end = tuple([vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]])
print(end)
stop1 = tuple([vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]])
print(stop1)
stop2 = tuple([vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]])
print(stop2)
stop3 = tuple([vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]])
print(stop3)


(-22.9661, -42.0278)
(-22.9661, -42.0278)
(-21.6403, -41.0511)
(-20.3297, -40.2925)
(-18.5933, -39.7322)


In [145]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')
fig.add_layer(vacation)
fig


Figure(layout=FigureLayout(height='420px'))

In [149]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,description,Lat,Lng,Hotel Name
0,219,Arraial Do Cabo,BR,75.07,clear sky,-22.9661,-42.0278,Pousada Porto Praia
1,298,Sao Joao Da Barra,BR,75.29,broken clouds,-21.6403,-41.0511,Pousada Porto De Canoas
2,532,Vila Velha,BR,75.15,few clouds,-20.3297,-40.2925,Quality Suites Vila Velha
3,661,Conceicao Da Barra,BR,75.33,few clouds,-18.5933,-39.7322,Cricaré Praia Hotel


In [150]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [151]:
# 4a. Add a marker layer for each city to the map.
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))